In [ ]:
import numpy as np
import scipy as sp


def WoS(DIM, epsilon, boundary, goal, start, obstacles, alpha, normalx = None):
    distB = DisttoBoundary(start, boundary)
    if distB < epsilon:
        return 0
 
    v = SampleSphere(DIM, distB, start)
    z = SampleBall(DIM, distB, start)
    s = 0
    if normalx and v.dot(normalx) > 0:
        v *= -1 #reflect v if it sampled point intersects same neumann that it is on
    
    while InstersectObstacle(z, obstacles):
        z = SampleBall(DIM, distB, start)

    gamma = sp.special.gamma(DIM)
    if z == goal:
        s = Greenfn(DIM, distB, gamma) * gamma * distB ** DIM

    a = PoissonKernel(DIM, distB, gamma, start, v)*gamma/DIM*distB**(DIM-1)
    if a < np.random.uniform(0, 1):
        return s

    intesectedObstacle = InstersectObstacle(v, obstacles)
    if intesectedObstacle:
        v, n = SampleObstacle(intesectedObstacle)
        return (a * WoS(DIM, epsilon, boundary, goal, v, obstacles, alpha, n) + s)/alpha

    return a * WoS(DIM, epsilon, boundary, goal, v, obstacles, alpha) + s




def DisttoBoundary(x, boundary):
    """Calculate the distance from point x to the given boundary."""

    pass

def InstersectObstacle(x, v, obstacles):
    """Check if instersects obstacle."""
    ##use bounding volume hierarchy to check for ray intersecction
    #not done
    pointsInCube = CheckPointsInCube(x, v, obstacles)
    if pointsInCube:
        return SplitCube(x, v, pointsInCube)
    return pointsInCube
 


def SplitCube(cube_max, cube_min, points):
    if max(cube_max - cube_min) < 1 or points == []:
        return points
    
    smaller_cube_max = (cube_max + cube_min) / 2
    smallerCubeIntersect = SplitCube(smaller_cube_max, cube_min, CheckPointsInCube(smaller_cube_max, cube_min, points))
    largerCubeIntersect = SplitCube(cube_max, smaller_cube_max, CheckPointsInCube(cube_max, smaller_cube_max, points))
    return smallerCubeIntersect + largerCubeIntersect



def CheckPointsInCube(cube_max, cube_min, points):
    pointsInCube = []

    for point in points:
        inCube = True
        for i in point:
            for j in range(len(cube_max)):
                if i<cube_min[j] or i>cube_max[j]:
                    inCube = False
                    break
            if not inCube:
                break
        if inCube:
            pointsInCube.append(point)
    return pointsInCube


def SampleObstacle(obstacles):
    """Sample a random point on the surface of a random obstacle."""
    #not done
    if not obstacles:
        return None
    obstacle = obstacles[np.random.randint(len(obstacles))]
    return obstacle  # Assuming obstacle is a point; modify as needed for actual shapes

def SampleSphere(DIM, r, center):
    """Sample a random point on the surface of a sphere."""
    vec = np.random.uniform(0, 1, DIM)
    vec /= np.linalg.norm(vec)
    return center + r * vec

def SampleBall(DIM, r, center):
    """Sample a random point inside a ball."""
    return center + np.random.uniform(0, r, DIM)

def PoissonKernel(DIM, r, gamma, x, v):
    return (x.dot(v)/np.linalg.norm(x)).dot(v-x)/(gamma*r**DIM)

def Greenfn(DIM, r, gamma):
    if DIM == 2:
        return -np.log(r)/gamma
    else:
        return -1/((DIM-2)*gamma*(r**(DIM-2)))



In [ ]:
"""code that might make it more efficient"""

def FieldofView(start, distB, obstacles):
    """Calculate the field of view from point x given the distance to boundary and obstacles."""
    #not done
    if not obstacles:
        return 4 * np.pi  # Full sphere in 3D
    fov = 4 * np.pi
    for obstacle in obstacles:
        dist = np.linalg.norm(start - obstacle)
        if dist < distB:
            angle = np.arccos(dist / distB)
            fov -= 2 * np.pi * (1 - np.cos(angle))
    return fov

def DisttoClosestObstacle(start, obstacles):
    """Calculate the distance from point x to the closest obstacle."""
    #not done
    if not obstacles:
        return np.inf
    min_dist = np.inf
    for obstacle in obstacles:
        dist = np.linalg.norm(start - obstacle)
        if dist < min_dist:
            min_dist = dist
    return min_dist

In [ ]:
# python program to implement Quick Hull algorithm
# to find convex hull.

# Stores the result (points of convex hull)
hull = set()

# Returns the side of point p with respect to line
# joining points p1 and p2.
def findSide(p1, p2, p):
    val = (p[1] - p1[1]) * (p2[0] - p1[0]) - (p2[1] - p1[1]) * (p[0] - p1[0])

    if val > 0:
        return 1
    if val < 0:
        return -1
    return 0

# returns a value proportional to the distance
# between the point p and the line joining the
# points p1 and p2
def lineDist(p1, p2, p):
    return abs((p[1] - p1[1]) * (p2[0] - p1[0]) -
            (p2[1] - p1[1]) * (p[0] - p1[0]))

# End points of line L are p1 and p2. side can have value
# 1 or -1 specifying each of the parts made by the line L
def quickHull(a, n, p1, p2, side):

    ind = -1
    max_dist = 0

    # finding the point with maximum distance
    # from L and also on the specified side of L.
    for i in range(n):
        temp = lineDist(p1, p2, a[i])
        
        if (findSide(p1, p2, a[i]) == side) and (temp > max_dist):
            ind = i
            max_dist = temp

    # If no point is found, add the end points
    # of L to the convex hull.
    if ind == -1:
        hull.add("$".join(map(str, p1)))
        hull.add("$".join(map(str, p2)))
        return

    # Recur for the two parts divided by a[ind]
    quickHull(a, n, a[ind], p1, -findSide(a[ind], p1, p2))
    quickHull(a, n, a[ind], p2, -findSide(a[ind], p2, p1))

def printHull(a, n):
    # a[i].second -> y-coordinate of the ith point
    if (n < 3):
        print("Convex hull not possible")
        return

    # Finding the point with minimum and
    # maximum x-coordinate
    min_x = 0
    max_x = 0
    for i in range(1, n):
        if a[i][0] < a[min_x][0]:
            min_x = i
        if a[i][0] > a[max_x][0]:
            max_x = i

    # Recursively find convex hull points on
    # one side of line joining a[min_x] and
    # a[max_x]
    quickHull(a, n, a[min_x], a[max_x], 1)

    # Recursively find convex hull points on
    # other side of line joining a[min_x] and
    # a[max_x]
    quickHull(a, n, a[min_x], a[max_x], -1)

    print("The points in Convex Hull are:")
    
    for element in hull:
        x = element.split("$")
        print("(", x[0], ",", x[1], ") ", end = " ")

# Driver code
a = [[0, 3], [1, 1], [2, 2], [4, 4],
     [0, 0], [1, 2], [3, 1], [3, 3]]
n = len(a)
printHull(a, n)

# The code is contributed by Nidhi goel